# 🎨 MangaGen MVP - Kaggle Pipeline

Generate a complete manga page (3 panels) from a text prompt using:
- **Gemini** for story → scene JSON
- **Stable Diffusion** for panel generation
- **OpenCV** for bubble placement
- **ReportLab** for PDF composition

---

## 🔧 Cell 1: Clone Repository

In [ ]:
%%bash
# Clone the MVP branch
git clone --branch mvp/kaggle-flux --single-branch https://github.com/Barun-2005/manga-gen-ai-pipeline.git manga-gen
cd manga-gen && git log -1 --oneline

## 📦 Cell 2: Install Dependencies

In [ ]:
# Install MVP requirements
!pip install -q -r manga-gen/requirements-mvp.txt

# Verify key imports
import google.generativeai as genai
import pydantic
from PIL import Image
print("✅ Dependencies installed successfully")

## 🔑 Cell 3: Set Up API Keys

### IMPORTANT: Add your GEMINI_API_KEY in Kaggle Secrets

1. Click the **Add-ons** menu at the top
2. Select **Secrets**
3. Click **+ Add a new secret**
4. Set **Label** to: `GEMINI_API_KEY`
5. Paste your API key in **Value**
6. Click **Save**

Get your Gemini API key from: https://makersuite.google.com/app/apikey

In [ ]:
import os
from kaggle_secrets import UserSecretsClient

# Load secrets from Kaggle
try:
    secrets = UserSecretsClient()
    os.environ['GEMINI_API_KEY'] = secrets.get_secret('GEMINI_API_KEY')
    print("✅ GEMINI_API_KEY loaded from Kaggle secrets")
except Exception as e:
    print(f"⚠️ Could not load secrets: {e}")
    print("Please add GEMINI_API_KEY in Kaggle Secrets (Add-ons → Secrets)")

# Optional: HuggingFace token for model downloads
try:
    os.environ['HF_TOKEN'] = secrets.get_secret('HF_TOKEN')
    print("✅ HF_TOKEN loaded")
except:
    print("ℹ️ HF_TOKEN not set (optional)")

## 📝 Cell 4: Generate Scene JSON (Gemini)

In [ ]:
import sys
sys.path.insert(0, 'manga-gen')
os.chdir('manga-gen')

# Define your story prompt
STORY_PROMPT = "A young ninja discovers an ancient temple hidden in the mountains and finds a glowing crystal sword"

# Run scene generation
!python scripts/generate_scene_json.py "{STORY_PROMPT}"

# Display the generated scene plan
import json
if os.path.exists('scene_plan.json'):
    with open('scene_plan.json') as f:
        scene_plan = json.load(f)
    print(f"\n✅ Generated {len(scene_plan)} scenes:")
    for scene in scene_plan:
        print(f"  - Panel {scene['panel']}: {scene.get('background', 'N/A')[:50]}...")
else:
    print("❌ scene_plan.json not found")

## 🎨 Cell 5: (Optional) Train LoRA for Character Consistency

Skip this cell for initial testing. Enable if you have character reference images.

In [ ]:
# Optional: Train LoRA if reference images exist
TRAIN_LORA = False  # Set to True to enable

if TRAIN_LORA and os.path.exists('character_refs/'):
    print("🎓 Training LoRA...")
    !python scripts/train_lora.py --refs character_refs/ --out artifacts/character.safetensors
    print("✅ LoRA training complete")
else:
    print("ℹ️ Skipping LoRA training (using IP-Adapter fallback)")

## 🖼️ Cell 6: Generate Panel Images

In [ ]:
# Create output directory
os.makedirs('outputs', exist_ok=True)

# Generate panels (uses --mock if GPU unavailable)
!python scripts/generate_panels.py --scene scene_plan.json --output outputs/

# Display generated panels
from IPython.display import display, Image as IPImage
import glob

panels = sorted(glob.glob('outputs/panel_*.png'))
print(f"\n✅ Generated {len(panels)} panels:")
for panel in panels:
    print(f"  - {panel}")
    display(IPImage(filename=panel, width=300))

## 💬 Cell 7: Place Dialogue Bubbles

In [ ]:
# Calculate bubble positions based on face detection
!python scripts/place_bubbles.py --panels outputs/ --scene scene_plan.json

# Display bubble data
if os.path.exists('bubbles.json'):
    with open('bubbles.json') as f:
        bubbles = json.load(f)
    print(f"\n✅ Generated bubble coordinates for {len(bubbles)} panels:")
    for item in bubbles:
        print(f"  - {item['panel']}: {len(item['bubbles'])} bubbles")
else:
    print("❌ bubbles.json not found")

## 📄 Cell 8: Compose Final PDF

In [ ]:
# Compose panels with bubbles into final PDF
!python scripts/compose_page.py --panels outputs/ --bubbles bubbles.json --output outputs/manga_page.pdf

# Verify output
if os.path.exists('outputs/manga_page.pdf'):
    print("\n✅ Manga page generated successfully!")
    print("📥 Download: outputs/manga_page.pdf")
    
    # Show file size
    size_mb = os.path.getsize('outputs/manga_page.pdf') / 1024 / 1024
    print(f"📦 File size: {size_mb:.2f} MB")
else:
    print("❌ PDF generation failed")

## 📦 Cell 9: Download Outputs

In [ ]:
%%bash
# Create downloadable zip of all outputs
cd outputs
zip -r ../manga_outputs.zip .
echo "\n✅ Created manga_outputs.zip"
ls -la ../manga_outputs.zip

In [ ]:
# Download link for Kaggle
from IPython.display import FileLink

os.chdir('/kaggle/working')
if os.path.exists('manga-gen/manga_outputs.zip'):
    import shutil
    shutil.copy('manga-gen/manga_outputs.zip', 'manga_outputs.zip')
    display(FileLink('manga_outputs.zip'))
    print("\n🎉 Click above to download your manga!")
else:
    print("⚠️ Output zip not found. Check previous cells for errors.")

---
## 📊 Summary

This notebook ran the complete MangaGen MVP pipeline:
1. ✅ Cloned repository (`mvp/kaggle-flux` branch)
2. ✅ Installed minimal dependencies
3. ✅ Generated scene plan using Gemini
4. ✅ (Optional) Trained LoRA for character consistency
5. ✅ Generated 3 panel images
6. ✅ Calculated bubble placements
7. ✅ Composed final PDF
8. ✅ Created downloadable outputs

**Output**: `outputs/manga_page.pdf` (3 panels with dialogue bubbles)